<a href="https://colab.research.google.com/github/Afsoon-Ghazanfari/Salary-Predictor/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics


In [3]:
df = pd.read_csv('/content/drive/My Drive/RediSchool-salarypredictor/survey_results_germany_modified_lastdata.csv')
df=df.drop(['Unnamed: 0'],axis=1)
df.head()

,YearsCodePro,CompTotal,"OrgSize_1,000 to 4,999 employees",OrgSize_10 to 19 employees,"OrgSize_10,000 or more employees",OrgSize_100 to 499 employees,OrgSize_2 to 9 employees,OrgSize_20 to 99 employees,"OrgSize_5,000 to 9,999 employees",OrgSize_500 to 999 employees,...,"EdLevel_Bachelor’s degree (B.A., B.S., B.Eng., etc.)","EdLevel_Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",EdLevel_Primary/elementary school,"EdLevel_Professional degree (JD, MD, Ph.D, Ed.D, etc.)","EdLevel_Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",EdLevel_Some college/university study without earning a degree,EdLevel_Something else,"RemoteWork_Hybrid (some remote, some in-person)",RemoteWork_In-person,RemoteWork_Remote
0,10,100000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,14,70044.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,5,80000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,10,65000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,7,50000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [4]:
X=df.drop(['CompTotal'],axis=1)
y=df['CompTotal']

In [5]:
# Normalization the data using the Standard Scaler
#standard=StandardScaler()
#X=standard.fit_transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state = 101)

In [ ]:
# Create the regressor
regressor = RandomForestRegressor()

# Define the hyperparameter grid
param_grid = {
    "n_estimators": [10, 50, 60],
    "max_depth": [None, 5, 10,20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5, 10, 20]
}

# Create the grid search object
grid_search = GridSearchCV(regressor, param_grid)

# Fit the grid search object to the data
grid_search.fit(X, y)

# Print the best parameters
print(grid_search.best_params_)

In [ ]:
# train the model using the best set of hyperparameters
randmf = RandomForestRegressor(max_depth=10, min_samples_leaf= 10, min_samples_split=2, n_estimators=100)
randmf.fit(X_train, y_train)

In [ ]:
y_pred_rf1 = pd.DataFrame( { "actual": y_test,
"predicted_prob": randmf.predict(
( X_test ) ) } )
y_pred_rf1

In [ ]:
mae = metrics.mean_absolute_error(y_pred_rf1['actual'], y_pred_rf1['predicted_prob'])
mse = metrics.mean_squared_error(y_pred_rf1['actual'], y_pred_rf1['predicted_prob'])
rmse = np.sqrt(mse) #mse**(0.5)
r2 = metrics.r2_score(y_pred_rf1['actual'], y_pred_rf1['predicted_prob'])
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

In [ ]:
from sklearn.model_selection import KFold
crossvalidation=KFold(n_splits=4,shuffle=True,random_state=1)


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

GBR=GradientBoostingRegressor()
search_grid={
    'n_estimators':[500,1000,2000],
    'learning_rate':[.001,0.01,.1],
    'max_depth':[1,2,4],
    'subsample':[.5,.75,1],
    'random_state':[1]
    }
search=GridSearchCV(estimator=GBR,param_grid=search_grid,scoring='neg_mean_squared_error',n_jobs=1,cv=crossvalidation)

In [ ]:
search.fit(X,y)
search.best_params_

In [ ]:
GBR2=GradientBoostingRegressor(n_estimators=500,learning_rate=0.01,subsample=.5,max_depth=1,random_state=1)
score=np.mean(cross_val_score(GBR2,X_train,y_train,scoring='neg_mean_squared_error',cv=crossvalidation,n_jobs=1))
score